In [1]:
#| default_exp functionsdb
import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

In [1]:
#| export
from pymongo import MongoClient
import pandas as pd
from decimal import Decimal, getcontext

# Funções de Banco

> Este módulo concentra as funções de banco de consultas na base de dados da Anatel no MongoDB.

In [2]:
# | export
def ConsultaSRD(
        mongo_client: MongoClient, # Objeto com o cliente para acesso aos dados do MongoDB                
) -> pd.DataFrame:    # DataFrame com os dados atualizados
    """Atualiza a tabela local dos dados de Radiodifusão do Mosaico"""

    # Colunas retornadas no mosaico
    MOSAICO_COLUMNS = (
        "Num_Serviço",
        "Id",
        "Número_Estação",
        "Latitude",
        "Longitude",
        "Validade_RF",
        "Município",
        "Frequência",
        "Classe",
        "Serviço",
        "Entidade",
        "UF",
        "Status",
        "CNPJ",
        "Fistel"
    )

    database = mongo_client["sms"]
    # Database com as informações de Radio e difusão
    collection = database["srd"]

    query = {}
    projection = {"SiglaServico": 1.0, "_id": 1.0, "Status.state": 1.0, "licensee": 1.0, "NumFistel": 1.0, "cnpj": 1.0, "frequency": 1.0, "stnClass": 1.0, "srd_planobasico.NomeMunicipio": 1.0, "srd_planobasico.SiglaUF": 1.0, "NumServico": 1.0, "estacao.NumEstacao": 1.0, "estacao.MedLatitudeDecimal": 1.0, "estacao.MedLongitudeDecimal": 1.0, "habilitacao.DataValFreq": 1.0}


    list_data = list(collection.find(query, projection = projection))
    mosaico_df = pd.json_normalize(list_data)
    mosaico_df = mosaico_df.drop(columns=['estacao'])
    mosaico_df = mosaico_df[["NumServico"
                            ,"_id"
                            ,"estacao.NumEstacao"
                            ,"estacao.MedLatitudeDecimal"
                            ,"estacao.MedLongitudeDecimal"
                            ,"habilitacao.DataValFreq"
                            ,"srd_planobasico.NomeMunicipio"
                            ,"frequency"
                            ,"stnClass"
                            ,"SiglaServico"
                            ,"licensee"
                            ,"srd_planobasico.SiglaUF"
                            ,"Status.state"
                            ,"cnpj"
                            ,"NumFistel"
    ]]

    mosaico_df.columns = MOSAICO_COLUMNS

    mosaico_df = mosaico_df[mosaico_df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)].reset_index(drop=True)

    for c in mosaico_df.columns:
        mosaico_df.loc[mosaico_df[c] == "", c] = pd.NA
    mosaico_df = mosaico_df.dropna(subset=['UF'])
    mosaico_df = mosaico_df[mosaico_df.Frequência.notna()].reset_index(drop=True)

    # mosaico_df = input_coordenates(mosaico_df, "../dados")
    # mosaico_df.loc["Frequência"] = mosaico_df.Frequência.str.replace(",", ".") 
    # mosaico_df.loc[:, "Frequência"] = mosaico_df.Frequência.astype("float")
    # mosaico_df.loc[mosaico_df.Serviço == "OM", "Frequência"] = mosaico_df.loc[
    #     mosaico_df.Serviço == "OM", "Frequência"
    # ].apply(lambda x: Decimal(x) / Decimal(1000))
    # mosaico_df.loc[:, "Validade_RF"] = mosaico_df.Validade_RF.astype("string").str.slice(0, 10)
    return mosaico_df    

In [ ]:
#| export
def consulta_telecom(mongo_client: MongoClient, # Objeto com o cliente para acesso aos dados do MongoDB                
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local dos dados de Radiodifusão do Mosaico"""

    database = mongo_client["sms"]
    collection = database["licenciamento"]

    query = {"DataExclusao" : None,
            "NumServico" : {u"$ne": u"010"},
            "Latitude" : {u"$ne": None},
            "Longitude": {u"$ne": None},
    }

    columns =  ['FreqTxMHz', 
                'CodTipoClasseEstacao', 
                'NumServico', 
                'NomeEntidade', 
                'NumFistel', 
                'NumEstacao', 
                'NomeMunicipio', 
                'CodMunicipio', 
                'SiglaUf', 
                'Latitude', 
                'Longitude', 
                'DataValidade', 
                'DesignacaoEmissao', 
                'Status.state']

    projection = {k:1.0 for k in columns}

    list_data = list(collection.find(query, projection = projection))
    df = pd.json_normalize(list_data)[columns]


In [ ]:
{
    "_id" : "6337604b0eb00",
    "NumFistel" : "50004466071",
    "NumServico" : "011",
    "NomeEntidade" : "TRACKER DO BRASIL LTDA",
    "SiglaUf" : "SP",
    "NumEstacao" : "505805553",
    "CodTipoClasseEstacao" : "FB",
    "NomeIndicativo" : "",
    "CodDebitoTFI" : "A",
    "DataPrimeiroLicenciamento" : "2000-03-14",
    "DataLicenciamento" : "2020-03-09",
    "Latitude" : "-23.4030555555555000",
    "Longitude" : "-46.6358333333333333",
    "NumRede" : "",
    "EnderecoEstacao" : "RUA LOS ANGELES                     .  BEVERLY HILL PA",
    "EndLogradouro" : "RUA LOS ANGELES",
    "EndNumero" : ".",
    "EndComplemento" : "",
    "EndBairro" : "",
    "CodCep" : "07600000",
    "NomeMunicipio" : "Mairiporã",
    "CodMunicipio" : "3528502",
    "NumAto" : "9992020",
    "DataValidade" : "2034-12-23",
    "FreqFinalMHz" : "173.07500000",
    "FreqTxMHz" : "173.07500000",
    "FreqRxMHz" : "173.07500000",
    "FreqInicialMHz" : "173.07500000",
    "GanhoAntena" : "5.8",
    "FrenteCostaAntena" : "",
    "AnguloMeiaPotenciaAntena" : "",
    "AnguloElevacao" : "",
    "Azimute" : "",
    "AlturaAntena" : "60.0",
    "CodTipoAntena" : 86,
    "CodProdutoAntena" : "",
    "CodEquipamentoAntena" : "",
    "Polarizacao" : "V",
    "CodProdutoTransmissor" : "33586",
    "PotenciaTransmissorWatts" : "150.000",
    "CodEquipamentoTransmissor" : "013051002112",
    "DesignacaoEmissao" : "13K2F2D--",
    "formId" : "base",
    "uid" : "6337604b0eb00",
    "Status" : {
        "state" : "LIC-LIC-01",
        "dateTime" : "2022-09-30 18:31:55",
        "user" : "sistema"
    },
    "state" : "LIC-LIC-01",
    "dateModified" : "2022-09-30 18:31:55",
    "data" : {
        "NumFistel" : "50004466071",
        "NumServico" : "011",
        "NomeEntidade" : "TRACKER DO BRASIL LTDA",
        "SiglaUf" : "SP",
        "NumEstacao" : "505805553",
        "stnclass" : "FB",
        "callsign" : "",
        "CodDebitoTFI" : "A",
        "DataPrimeiroLicenciamento" : "2000-03-14",
        "DataLicenciamento" : "2020-03-09",
        "latitude" : "-23.4030555555555000",
        "longitude" : "-46.6358333333333333",
        "NumRede" : "",
        "EnderecoEstacao" : "RUA LOS ANGELES                     .  BEVERLY HILL PA",
        "EndLogradouro" : "RUA LOS ANGELES",
        "EndNumero" : ".",
        "EndComplemento" : "",
        "EndBairro" : "",
        "CodCep" : "07600000",
        "NomeMunicipio" : "Mairiporã",
        "CodMunicipio" : "3528502",
        "NumAto" : "9992020",
        "DataValidade" : "2034-12-23",
        "txfreq" : "173.07500000",
        "rxfreq" : "173.07500000",
        "gain" : "5.8",
        "ftbr" : "",
        "beamwidth" : "",
        "tilt" : "",
        "azimuth" : "",
        "antheight" : "60.0",
        "CodTipoAntena" : "086",
        "CodProdutoAntena" : "",
        "CodEquipamentoAntena" : "",
        "polar" : "V",
        "CodProdutoTransmissor" : "33586",
        "pwr" : "150.000",
        "CodEquipamentoTransmissor" : "013051002112",
        "ems" : "13K2F2D--",
        "formId" : "base",
        "uid" : "6337604b0eb00",
        "state" : "LIC-LIC-01",
        "dateCreated" : "2022-09-30 18:31:55",
        "dateModified" : "2022-09-30 18:31:55",
        "usr" : "sistema"
    }
}